In [13]:
import mne
import matplotlib 
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
from mne.preprocessing import ICA
from mne.preprocessing import read_ica
raw = mne.io.read_raw_fif('rEEG\sub-032\preICA032_raw.fif', preload=True)
ica = read_ica(f'rEEG\sub-032\ica-032-variance_compica.fif')

Opening raw data file rEEG\sub-032\preICA032_raw.fif...
    Range : 0 ... 37899 =      0.000 ...   151.596 secs
Ready.
Reading 0 ... 37899  =      0.000 ...   151.596 secs...
Reading c:\Users\User\Documents\EEG_Project\rEEG\sub-032\ica-032-variance_compica.fif ...
Now restoring ICA solution ...
Ready.


C:\Users\User\AppData\Local\Temp\ipykernel_38092\1323802612.py:8: RuntimeWarning: This filename (rEEG\sub-032\ica-032-variance_compica.fif) does not conform to MNE naming conventions. All ICA files should end with -ica.fif, -ica.fif.gz, _ica.fif or _ica.fif.gz
  ica = read_ica(f'rEEG\sub-032\ica-032-variance_compica.fif')


In [14]:
# ---------------------------------------------------------
# CONSERVATIVE MUSCLE ARTIFACT DETECTOR FOR ICA (plug-and-play)
# ---------------------------------------------------------
import numpy as np
import pandas as pd
from scipy import signal, stats


def detect_muscle_artifacts(ica, raw):
    """
    Conservative automatic muscle artifact detection.
    INPUTS:
        ica : your fitted ICA object
        raw : the Raw you used for ICA
    OUTPUTS:
        df  : table ranking muscle-like ICs
        bad : list of IC indices to remove
    """

    sfreq = raw.info['sfreq']
    sources = ica.get_sources(raw).get_data()     # (n_ics, n_times)
    mixing = np.abs(ica.get_components())         # (n_channels, n_ics)

    emg_band = (20, 45)

    rows = []
    for ic in range(sources.shape[0]):
        sig = sources[ic]

        # --- PSD ---
        freqs, pxx = signal.welch(sig, sfreq, nperseg=int(2*sfreq))
        emg_mask = (freqs >= emg_band[0]) & (freqs <= emg_band[1])

        emg_pow = np.trapz(pxx[emg_mask], freqs[emg_mask])
        low_pow = np.trapz(pxx[(freqs>=1)&(freqs<=15)], freqs[(freqs>=1)&(freqs<=15)])
        emg_ratio = emg_pow / (low_pow + 1e-12)

        # --- Spike / burst detection ---
        zsig = (sig - sig.mean()) / (sig.std() + 1e-12)
        burst_count = np.sum(np.abs(zsig) > 3)

        # --- Spatial focality ---
        focality = mixing[:, ic].max() / (np.median(mixing[:, ic]) + 1e-12)

        # --- Conservative scoring ---
        score = (
            0.45 * min(emg_ratio / 3, 1) +      # harder to score high
            0.25 * min(burst_count / 80, 1) +   # requires more bursts
            0.30 * min(focality / 8, 1)         # requires tighter focality
        )

        rows.append({
            'IC': ic,
            'EMG_ratio': emg_ratio,
            'bursts': int(burst_count),
            'focality': focality,
            'score': score
        })

    df = pd.DataFrame(rows).sort_values('score', ascending=False).reset_index(drop=True)

    # Conservative threshold: score > 0.75  
    bad = df[df['score'] > 0.75]['IC'].tolist()

    return df, bad


df, bad_ics = detect_muscle_artifacts(ica, raw)

print(df.head(50).to_string(index=False))
print("ICs to remove:", bad_ics)


C:\Users\User\AppData\Local\Temp\ipykernel_38092\3255221046.py:34: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  emg_pow = np.trapz(pxx[emg_mask], freqs[emg_mask])
C:\Users\User\AppData\Local\Temp\ipykernel_38092\3255221046.py:35: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  low_pow = np.trapz(pxx[(freqs>=1)&(freqs<=15)], freqs[(freqs>=1)&(freqs<=15)])


 IC  EMG_ratio  bursts  focality    score
 22   2.738711     293  5.782181 0.877639
 53   0.788825     176  7.675130 0.656141
 31   0.665942     246  8.604419 0.649891
 37   0.633809     403 11.654019 0.645071
 39   0.970139     153  6.498875 0.639229
 35   0.439477     372 16.806697 0.615922
 34   0.372610     462 18.950940 0.605892
 47   0.349354     345 19.804229 0.602403
 41   0.318402     280 11.729165 0.597760
 30   0.680359     251  6.454518 0.594098
 10   0.211853     439  7.834960 0.575589
 50   0.155873     420 16.740175 0.573381
 45   0.153033     437 21.150758 0.572955
 40   0.145426     345 10.212235 0.571814
 54   1.005743     175  4.506053 0.569838
 42   0.119736     482 11.110566 0.567960
  3   0.103847     520 16.379014 0.565577
 49   0.178642     333  7.668479 0.564364
 36   0.046174     531 29.316004 0.556926
 51   0.291425     564  6.940012 0.553964
 43   1.094727     141  2.721319 0.516258
 21   0.204741     370  5.786569 0.497708
 46   0.418251     230  4.758224 0

In [7]:
# ---------------------------------------------------------
# EASY EOG/BLINK ARTIFACT DETECTOR FOR ICA (plug-and-play)
# ---------------------------------------------------------
import numpy as np
import pandas as pd
from scipy import signal

def detect_eog_artifacts(ica, raw):
    """
    Simple automatic EOG/blink artifact detection.
    INPUTS:
        ica : your fitted ICA object
        raw : the Raw you used for ICA
    OUTPUTS:
        df  : table ranking EOG-like ICs
        bad : list of IC indices to remove
    """
    
    sfreq = raw.info['sfreq']
    sources = ica.get_sources(raw).get_data()     # shape: (n_ics, n_times)
    mixing = np.abs(ica.get_components())         # shape: (n_channels, n_ics)
    ch_names = [ch['ch_name'] for ch in raw.info['chs']]
    
    # frontal electrodes typically showing EOG
    frontal_chs = ['Fp1', 'Fp2', 'AF7', 'AF8', 'AF3', 'AF4', 'Fz']
    frontal_idx = [ch_names.index(ch) for ch in frontal_chs if ch in ch_names]
    
    eog_band = (0.5, 6)  # blink / slow eye movement
    
    rows = []
    for ic in range(sources.shape[0]):
        sig = sources[ic]
        
        # --- PSD in low-frequency range ---
        freqs, pxx = signal.welch(sig, sfreq, nperseg=int(2*sfreq))
        eog_mask = (freqs >= eog_band[0]) & (freqs <= eog_band[1])
        eog_pow = np.trapz(pxx[eog_mask], freqs[eog_mask])
        total_pow = np.trapz(pxx, freqs)
        eog_ratio = eog_pow / (total_pow + 1e-12)
        
        # --- Spike / burst detection ---
        zsig = (sig - sig.mean()) / (sig.std() + 1e-12)
        burst_count = np.sum(np.abs(zsig) > 3)
        
        # --- Frontal dominance ---
        frontal_weight = mixing[frontal_idx, ic].mean()
        overall_weight = mixing[:, ic].mean()
        frontal_ratio = frontal_weight / (overall_weight + 1e-12)
        
        # --- Score ---
        # Weighting: frontal_ratio is primary, then low-freq power, then bursts
        score = (
            0.8 * min(frontal_ratio / 2, 1) +
            0.3 * min(eog_ratio / 0.5, 1) +
            0.1 * min(burst_count / 50, 1)
        )
        
        rows.append({
            'IC': ic,
            'frontal_ratio': frontal_ratio,
            'eog_ratio': eog_ratio,
            'bursts': int(burst_count),
            'score': score
        })
        
    df = pd.DataFrame(rows).sort_values('score', ascending=False).reset_index(drop=True)
    
    # Threshold: score > 0.5 → probably EOG
    bad = df[df['score'] > 1.1]['IC'].tolist()
    
    return df, bad

# ------------------------------
# Example usage / printing
# ------------------------------
df, bad_ics = detect_eog_artifacts(ica, raw)

print(df.head(50).to_string(index=False))
print("ICs to remove:", bad_ics)


C:\Users\User\AppData\Local\Temp\ipykernel_38092\3021004265.py:37: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  eog_pow = np.trapz(pxx[eog_mask], freqs[eog_mask])
C:\Users\User\AppData\Local\Temp\ipykernel_38092\3021004265.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  total_pow = np.trapz(pxx, freqs)


 IC  frontal_ratio  eog_ratio  bursts    score
  0       3.739371   0.729583    1183 1.200000
  4       2.068278   0.580722     616 1.200000
 49       1.786899   0.460640     368 1.091143
 17       2.066923   0.251383     160 1.050830
 41       1.698778   0.445667     258 1.046912
 23       1.844813   0.333015     182 1.037734
 46       1.976053   0.186642     173 1.002406
 30       2.945789   0.155214     289 0.993129
  9       2.013603   0.115118     313 0.969071
 24       1.991766   0.117149     388 0.966996
 45       1.513395   0.410154     256 0.951451
 21       4.197827   0.084700     264 0.950820
  5       1.689277   0.275621     233 0.941084
  3       3.142982   0.025106     701 0.915063
 18       1.789205   0.159639     159 0.911465
 33       1.634538   0.260142     184 0.909900
 40       1.174283   0.529172     441 0.869713
 12       1.713317   0.103743     377 0.847572
  1       1.552143   0.163953     796 0.819229
 35       1.284709   0.335389     451 0.815117
 53       1.1

In [3]:
import numpy as np
from mne.time_frequency import psd_array_welch

# Example: components 1 through 37 inclusive
ic_range = range(1,48)  

sources = ica.get_sources(raw).get_data()  # shape: n_components x n_times

print(f"{'IC':>4} | {'HF/LF Ratio (30-40/1-30 Hz)':>25}")
print("-"*32)

for comp in ic_range:
    comp_ts = sources[comp]  # 1D array of this IC's time series
    psd, freqs = psd_array_welch(comp_ts, sfreq=raw.info['sfreq'], fmin=1, fmax=40, n_fft=2048)
    
    low = psd[(freqs >= 1) & (freqs < 30)].mean()
    high = psd[(freqs >= 30) & (freqs <= 40)].mean()
    ratio = high / low if low > 0 else np.nan
    
    print(f"{comp:>4} | {ratio:>25.4f}")


  IC | HF/LF Ratio (30-40/1-30 Hz)
--------------------------------
Effective window size : 8.192 (s)


   1 |                    1.2694
Effective window size : 8.192 (s)
   2 |                    0.1478
Effective window size : 8.192 (s)
   3 |                    0.2560
Effective window size : 8.192 (s)
   4 |                    0.4290
Effective window size : 8.192 (s)
   5 |                    0.2435
Effective window size : 8.192 (s)
   6 |                    0.3448
Effective window size : 8.192 (s)
   7 |                    1.3383
Effective window size : 8.192 (s)
   8 |                    0.3851
Effective window size : 8.192 (s)
   9 |                    0.6083
Effective window size : 8.192 (s)
  10 |                    2.2213
Effective window size : 8.192 (s)
  11 |                    0.7162
Effective window size : 8.192 (s)
  12 |                    0.8081
Effective window size : 8.192 (s)
  13 |                    0.7409
Effective window size : 8.192 (s)
  14 |                    0.7653
Effective window size : 8.192 (s)
  15 |                    0.9397
Effective window size : 8.192

In [13]:
import numpy as np
from mne.time_frequency import psd_array_welch

# 1) channel HF power (you already ran this)
data = raw.get_data()
sfreq = raw.info['sfreq']
psds_ch, freqs = psd_array_welch(data, sfreq=sfreq, fmin=1, fmax=80, n_fft=2048, n_overlap=0)
hf_idx = (freqs >= 30) & (freqs <= 45)
ch_hf = psds_ch[:, hf_idx].mean(axis=1)         # V^2/Hz per channel
ch_hf_uv = ch_hf * 1e12                         # µV^2/Hz (readable)

# 2) component HF power (show why ICA components look 'hot')
sources = ica.get_sources(raw).get_data()       # n_ics x n_times
psds_ic, freqs_ic = psd_array_welch(sources, sfreq=sfreq, fmin=1, fmax=80, n_fft=2048, n_overlap=0)
ic_hf = psds_ic[:, hf_idx].mean(axis=1)         # V^2/Hz per IC
ic_hf_uv = ic_hf * 1e12

# 3) relative contribution: sum HF power of ICs that you flagged vs total channel HF
total_ch_hf = ch_hf.sum()
total_ic_hf = ic_hf.sum()                       # should be comparable (same energy space)
print("Total channel HF (V^2/Hz):", total_ch_hf)
print("Total IC HF (V^2/Hz):     ", total_ic_hf)
print("Ratio IC/Channel total:", total_ic_hf / (total_ch_hf + 1e-30))

# 4) how much HF power is removed if you drop candidate muscle ICs (example list)
candidates = [20,16,47]   # replace with your auto_reject list
# project data without those ICs
ica.exclude = candidates
recon = ica.apply(raw.copy(), exclude=candidates, start=None, stop=None).get_data()
psds_recon, _ = psd_array_welch(recon, sfreq=sfreq, fmin=1, fmax=80, n_fft=2048, n_overlap=0)
ch_hf_recon = psds_recon[:, hf_idx].mean(axis=1)
print("Median channel HF before (µV^2/Hz):", np.median(ch_hf_uv))
print("Median channel HF after  (µV^2/Hz):", np.median(ch_hf_recon * 1e12))


Effective window size : 8.192 (s)
Effective window size : 8.192 (s)
Total channel HF (V^2/Hz): 2.1388942086440526e-11
Total IC HF (V^2/Hz):      2.2269573301745
Ratio IC/Channel total: 104117226610.57999
Applying ICA to Raw instance
    Transforming to ICA space (49 components)
    Zeroing out 3 ICA components
    Projecting back using 60 PCA components
Effective window size : 8.192 (s)
Median channel HF before (µV^2/Hz): 0.3096641917078896
Median channel HF after  (µV^2/Hz): 0.30681398793386216
